# 📊 Анализ рецептов терразитовой штукатурки

Этот ноутбук анализирует базу рецептов терразитовой штукатурки из Excel файла.

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from pathlib import Path

# Настройка стилей графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 2. Загрузка данных из Excel

In [ ]:
# Путь к Excel файлу
excel_path = Path('../data/raw/Рецептуры терразит.xlsx')

if not excel_path.exists():
    print("❌ Файл не найден. Убедитесь, что файл находится в data/raw/")
else:
    print(f"✅ Файл найден: {excel_path.name}")
    print(f"📏 Размер файла: {excel_path.stat().st_size / 1024:.1f} KB")

In [ ]:
# Загрузка Excel файла
try:
    df = pd.read_excel(excel_path, sheet_name=0)
    print(f"✅ Данные успешно загружены")
    print(f"📊 Размер таблицы: {df.shape[0]} строк, {df.shape[1]} столбцов")
    
    # Показать первые 5 строк
    print("\nПервые 5 строк данных:")
    display(df.head())
    
except Exception as e:
    print(f"❌ Ошибка при загрузке Excel: {e}")

## 3. Очистка и подготовка данных

In [ ]:
# Очистка названий столбцов
df.columns = [str(col).strip().replace('\n', ' ') for col in df.columns]

# Найдем столбец с названиями рецептов
recipe_col = None
for col in df.columns:
    if 'наименование' in col.lower() or 'рецепт' in col.lower():
        recipe_col = col
        break

if recipe_col:
    print(f"✅ Столбец с рецептами: {recipe_col}")
    
    # Создаем список рецептов
    recipes = []
    
    for idx, row in df.iterrows():
        recipe_name = row[recipe_col]
        
        if pd.isna(recipe_name):
            continue
            
        recipe = {
            'id': f"REC_{idx:03d}",
            'name': str(recipe_name).strip(),
            'type': 'терразит' if 'терразит' in str(recipe_name).lower() else 'шовный',
            'components': {}
        }
        
        # Собираем компоненты
        for col in df.columns:
            if col == recipe_col:
                continue
                
            value = row[col]
            if pd.isna(value):
                value = 0
            
            try:
                value = float(value)
            except:
                value = 0
            
            if value > 0:
                recipe['components'][col] = value
        
        recipes.append(recipe)
    
    print(f"📝 Извлечено рецептов: {len(recipes)}")
    print(f"🏗️  Терразитов: {sum(1 for r in recipes if r['type'] == 'терразит')}")
    print(f"🧱 Шовных: {sum(1 for r in recipes if r['type'] == 'шовный')}")

else:
    print("❌ Не найден столбец с названиями рецептов")

## 4. Анализ компонентов

In [ ]:
# Анализ всех компонентов
all_components = {}
for recipe in recipes:
    for component, weight in recipe['components'].items():
        if weight > 0:
            all_components[component] = all_components.get(component, 0) + weight

print(f"📦 Всего уникальных компонентов: {len(all_components)}")
print(f"🏆 Топ-5 самых используемых компонентов:")

# Сортируем по использованию
sorted_components = sorted(all_components.items(), key=lambda x: x[1], reverse=True)

for i, (comp, total) in enumerate(sorted_components[:5], 1):
    print(f"{i}. {comp[:40]}... : {total:.1f} кг")

## 5. Визуализация данных

In [ ]:
# Создадим DataFrame для анализа
analysis_data = []
for recipe in recipes:
    analysis_data.append({
        'Тип': recipe['type'],
        'Название': recipe['name'],
        'Кол-во компонентов': len(recipe['components']),
        'Общий вес': sum(recipe['components'].values())
    })

analysis_df = pd.DataFrame(analysis_data)

# Показать статистику
print("📊 Статистика по рецептам:")
display(analysis_df.describe())

print("\n📋 По типам рецептов:")
display(analysis_df.groupby('Тип').agg({
    'Кол-во компонентов': ['mean', 'min', 'max'],
    'Общий вес': ['mean', 'min', 'max']
}))

In [ ]:
# График 1: Распределение типов рецептов
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Круговая диаграмма типов
type_counts = analysis_df['Тип'].value_counts()
axes[0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[0].set_title('Распределение типов рецептов')

# Гистограмма количества компонентов
axes[1].hist(analysis_df['Кол-во компонентов'], bins=15, edgecolor='black')
axes[1].set_xlabel('Количество компонентов в рецепте')
axes[1].set_ylabel('Частота')
axes[1].set_title('Распределение количества компонентов')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# График 2: Топ-15 компонентов
top_15 = sorted_components[:15]
components_names = [c[0][:30] + '...' if len(c[0]) > 30 else c[0] for c in top_15]
components_values = [c[1] for c in top_15]

plt.figure(figsize=(12, 8))
bars = plt.barh(components_names, components_values, color='skyblue')
plt.xlabel('Общий вес использования (кг)', fontsize=12)
plt.title('Топ-15 компонентов по использованию', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # Перевернуть для лучшей читаемости

# Добавить значения на график
for bar in bars:
    width = bar.get_width()
    plt.text(width + 10, bar.get_y() + bar.get_height()/2, 
             f'{width:.0f}', ha='left', va='center')

plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# График 3: Сравнение терразитов и шовных
terrazite_df = analysis_df[analysis_df['Тип'] == 'терразит']
seam_df = analysis_df[analysis_df['Тип'] == 'шовный']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Количество компонентов
axes[0].boxplot([terrazite_df['Кол-во компонентов'], seam_df['Кол-во компонентов']], 
                labels=['Терразиты', 'Шовные'])
axes[0].set_ylabel('Количество компонентов')
axes[0].set_title('Сравнение количества компонентов')
axes[0].grid(True, alpha=0.3)

# Общий вес
axes[1].boxplot([terrazite_df['Общий вес'], seam_df['Общий вес']], 
                labels=['Терразиты', 'Шовные'])
axes[1].set_ylabel('Общий вес (кг)')
axes[1].set_title('Сравнение общего веса')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Сохранение результатов анализа

In [ ]:
# Сохраняем структурированные данные
output_dir = Path('../data/processed')
output_dir.mkdir(exist_ok=True)

# Сохраняем JSON
with open(output_dir / 'recipes_analysis.json', 'w', encoding='utf-8') as f:
    json.dump({
        'recipes': recipes,
        'statistics': {
            'total_recipes': len(recipes),
            'total_components': len(all_components),
            'terrazite_count': sum(1 for r in recipes if r['type'] == 'терразит'),
            'seam_count': sum(1 for r in recipes if r['type'] == 'шовный'),
            'top_components': sorted_components[:10]
        }
    }, f, ensure_ascii=False, indent=2)

print(f"✅ Данные анализа сохранены: {output_dir / 'recipes_analysis.json'}")

# Сохраняем графики
figures_dir = output_dir / 'figures'
figures_dir.mkdir(exist_ok=True)

# Сохраняем последний график как пример
plt.figure(figsize=(10, 6))
bars = plt.barh([c[0][:25] + '...' for c in top_15], components_values)
plt.title('Топ-15 компонентов терразитовой штукатурки')
plt.xlabel('Общий вес (кг)')
plt.tight_layout()
plt.savefig(figures_dir / 'top_components.png', dpi=150, bbox_inches='tight')
plt.close()

print(f"✅ Графики сохранены: {figures_dir}")

## 7. Выводы и рекомендации

In [ ]:
# Генерация отчета
print("=" * 60)
print("📊 ОТЧЕТ ПО АНАЛИЗУ РЕЦЕПТОВ ТЕРРАЗИТОВОЙ ШТУКАТУРКИ")
print("=" * 60)
print(f"\n📈 ОБЩАЯ СТАТИСТИКА:")
print(f"• Всего рецептов: {len(recipes)}")
print(f"• Терразитов: {sum(1 for r in recipes if r['type'] == 'терразит')}")
print(f"• Шовных: {sum(1 for r in recipes if r['type'] == 'шовный')}")
print(f"• Уникальных компонентов: {len(all_components)}")

print(f"\n🏆 ТОП-3 КОМПОНЕНТА:")
for i, (comp, total) in enumerate(sorted_components[:3], 1):
    print(f"  {i}. {comp} - {total:.0f} кг")

print(f"\n🎯 РЕКОМЕНДАЦИИ ДЛЯ ML МОДЕЛИ:")
print("1. Фокус на топ-20 компонентов (охват >95% состава)")
print("2. Создать категориальные признаки по типам заполнителей")
print("3. Нормализовать данные по общему весу (1000 кг)")
print("4. Разделить на терразиты и шовные для отдельных моделей")

print("\n📁 Результаты сохранены в папке: data/processed/")